In [2]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

In [8]:
product_descriptions = pd.read_csv('/home/MZ/Documents/CODE/THINKFUL/projects/kaggle/product_descriptions.csv', header=0)
attributes = pd.read_csv('/home/MZ/Documents/CODE/THINKFUL/projects/kaggle/attributes.csv', header=0)
train = pd.read_csv('/home/MZ/Documents/CODE/THINKFUL/projects/kaggle/train.csv', header=0,  encoding = "ISO-8859-1")

# FEATURES

### EXACT MATCH, between search term (as phrase) and product title, allowing only differences in case

In [11]:
# A boolean

train['exact'] = pd.Series([True if st.lower() in pt.lower() else False 
                      for st, pt in zip(train['search_term'], train['product_title'])])

### OVERLAPPING WORDS, allowing for differences in case

In [13]:
# A count >= 0

train['overlapping_words'] = pd.Series([len(set(st.lower().split()) & set(pt.lower().split()))
                      for st, pt in zip(train['search_term'], train['product_title'])])

### PERCENTAGE OVERLAPPING WORDS, allowing for differences in case

In [59]:
train['percentage_overlapping_words'] = pd.Series([len(set(st.lower().split()) & 
                                                       set(pt.lower().split()))/float(len(st.split()))
                                                       for st, pt in zip(train['search_term'], train['product_title'])])

### DEFAULT PENALTY: products that appear too many times -- and have low relevance -- suggesting that they are defaults for when the search engine is stumped

In [18]:
# Aggregate on product ID's themselves and penalize too popular products.
from collections import Counter
d = Counter(train['product_uid'])
for pid in d: 
    if d[pid] > 5:
        pass
#         print(pid, d[pid],) 

### BAG OF WORDS: product descriptions, attributes, titles -- product by product

In [62]:
print(len(product_descriptions), len(attributes), len(train))

124428 2044803 74067


In [19]:
# TF-IDF on product description
text = ''
len(product_descriptions['product_description']) # 124428
# for d in product_descriptions['product_description']:
#     text+=d
x = ''.join(product_descriptions['product_description']).split()
d = Counter(x)
frequent = sorted(d.items(), key=lambda x: x[1])[:50:-1]

In [20]:
frequent[:10]

[('and', 632164),
 ('to', 379616),
 ('the', 354414),
 ('for', 332252),
 ('a', 315015),
 ('with', 239997),
 ('of', 229611),
 ('is', 191992),
 ('in.', 186786),
 ('in', 177073)]

In [21]:
d = {w[0]: 0 for w in frequent}

In [47]:
# Bag of words attributes

# (100001.0, 'Bullet01', 'Versatile connector for various 90° connections and home repair projects')

attribute_bag = {}
for row in zip(train['product_uid'], attributes['name'], attributes['value']):
#     print (row)
    product_uid = int(row[0])
    if product_uid in attribute_bag:
        
        attribute_bag[product_uid] = ' ` '.join([attribute_bag[product_uid], str(row[-1])])
    else:
        attribute_bag[product_uid] = str(row[-1])

In [53]:
print(len(attribute_bag))

54667


In [56]:
print(len([x for x in zip(train['product_uid'], train['product_title'], product_descriptions['product_description'])]))


74067


In [51]:
for row in zip(train['product_uid'], train['product_title'], product_descriptions['product_description']):
    attribute_bag[row[0]] = ' ~ '.join([row[1], row[2], attribute_bag[row[0]]])

In [49]:
N = len(train)

74067

In [33]:
# What do 3's have in common?

In [ ]:
# What do 1's have in common?